test for doesthedogdie API

Imports

In [ ]:
import json
import re
import requests
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# API request
url = 'https://www.doesthedogdie.com/media/9593'
headers = {
    'Accept': 'application/json',
    'X-API-KEY': '8bf1393430a063dd2b27f3699f1625bc'
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()

    # Initialize the result structure to store the final extracted data
    result = {
        'id': data['item'].get('id', 'Unknown'),
        'name': data['item'].get('name', 'Unknown'),
        'cleanName': data['item'].get('cleanName', 'Unknown'),
        'genre': data['item'].get('genre', 'Unknown'),
        'releaseYear': data['item'].get('releaseYear', 'Unknown'),
        'triggers': []  # store all triggers
    }

    # Iterate over topicItemStats to collect triggers
    for topic_item in data.get('topicItemStats', []):
        # Extract the relevant information from topic and topicItemStats
        topic = topic_item.get('topic', {})
        if topic:
            does_name = topic.get('doesName', 'Unknown')
            yes_sum = topic_item.get('yesSum', 0)
            no_sum = topic_item.get('noSum', 0)

            # Append trigger information to list in the result
            result['triggers'].append({
                'doesName': does_name,
                'yesSum': yes_sum,
                'noSum': no_sum
            })

    # print structured result
    print(json.dumps(result, indent=4))

else:
    print(f'Error: {response.status_code} - {response.text}')


Flatten Data, extract information and generate .csv file

In [ ]:
if response.status_code == 200:
    data = response.json()

    # Initialize a list to store data for the DataFrame
    rows = []

    # Extract the main item data (id, name, cleanName, genre, releaseYear)
    item_data = {
        'id': data['item'].get('id', 'Unknown'),
        'name': data['item'].get('name', 'Unknown'),  # cleanName formatted as name
        'cleanName': data['item'].get('cleanName', 'Unknown').lower().replace(' ', '_'),
        'genre': data['item'].get('genre', 'Unknown'),
        'releaseYear': data['item'].get('releaseYear', 'Unknown'),
    }

    # Iterate over the topicItemStats to collect triggers and add rows
    for topic_item in data.get('topicItemStats', []):
        topic = topic_item.get('topic', {})
        if topic:
            does_name = topic.get('doesName', 'Unknown')
            yes_sum = topic_item.get('yesSum', 0)
            no_sum = topic_item.get('noSum', 0)

            # Add a row to the list of rows with the relevant data
            rows.append({
                'id': item_data['id'],
                'name': item_data['name'],  # Automatically formatted cleanName as name
                'cleanName': item_data['cleanName'],
                'genre': item_data['genre'],
                'releaseYear': item_data['releaseYear'],
                'doesName': does_name,
                'yesSum': yes_sum,
                'noSum': no_sum
            })

    # Create the DataFrame from the rows
    df = pd.DataFrame(rows)

    # Create the CSV file name based on the cleanName (lowercase with underscores)
    file_name = f'{item_data['cleanName'] + '_df'}.csv'

    # Save the DataFrame to a CSV file
    df.to_csv(f'./data/raw/{file_name}', index=False)

    # Print confirmation message
    print(f'Data saved to {file_name}')

else:
    print(f'Error: {response.status_code} - {response.text}')